## ETLGanz Get Flights
### 1. Objective
To fetch flight data using the AeroDataBox (Rapid API), set up dataframes, and use SQL to store data for table queries. This notebook will focus on retrieving real-time flight information, processing the data, and storing it in a structured format for SQL Queries, further analysis as needed. 



### 2. Global Configuration

In [14]:
# Global Configuration
# Import necessary libraries

import pandas as pd
import numpy as np
import requests as req
import sqlalchemy as sa
import pymysql as pms
import re
import myconfig as cfg

from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from pytz import timezone

import warnings

warnings.filterwarnings('ignore')

#Default path for data
# path = "..\Data\\"

###  3. Third Party Data
#####
URLs for Airport, Flights details are below:
```markdown
- AeroDataBox
- [RapidAI](https://rapidapi.com/aedbx-aedbx/api/aerodatabox)

```

In [2]:
url1 ="https://aerodatabox.p.rapidapi.com/airports/search/location"

### 4. Flights
### 4.1 Initial Checks
##### Parameters
- **Site Response**: Check
- **Lat, Long Definitions**: Defining the Latitudes, Longitudes for the chosen city
- **Data Extraction Checks**: for city name, country, icao, iata, timezone data etc.

In [3]:
response1 = req.get(url1)
response1.status_code

401

In [46]:
# Hash individually to check API Functionality
# Define Lat, Long for Berlin
latitudes = [52.5200]
longitudes = [13.4050]

# Define Lat, Long for Hamburg
# latitudes = [53.5511]
# longitudes = [9.9937]

# Define Lat, Long for Munich
# latitudes = [48.1351]
# longitudes = [11.5820]

In [ ]:
querystring = {"lat":latitudes,"lon":longitudes,"radiusKm":"50","limit":"2","withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": "Input", # Insert your RapidAPI Key
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = req.get(url1, headers=headers, params=querystring)

response.json()

{'searchBy': {'lat': 52.52, 'lon': 13.405},
 'count': 1,
 'items': [{'icao': 'EDDB',
   'iata': 'BER',
   'name': 'Berlin Brandenburg',
   'shortName': 'Brandenburg',
   'municipalityName': 'Berlin',
   'location': {'lat': 52.35139, 'lon': 13.493889},
   'countryCode': 'DE',
   'timeZone': 'Europe/Berlin'}]}

In [ ]:
#Create dataframe with ICAO codes
def icao_airport_codes(latitudes, longitudes):

    list_for_df = []

    for index, value in enumerate(latitudes):

        querystring = {"lat":value,"lon":longitudes[index],"radiusKm":"50","limit":"3","withFlightInfoOnly":"true"}
        headers = {
            "X-RapidAPI-Key": "Input",  # Insert your RapidAPI Key
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = req.get(url1, headers=headers, params=querystring)
        list_for_df.append(pd.json_normalize(response.json()['items']))

    return pd.concat(list_for_df, ignore_index=True)

# coordinates for Berlin, 'Hamburg', 'Munich'
latitudes = [52.5200, 53.5511, 48.1351]
longitudes = [13.4050, 9.9937, 11.5820]

icao_airport_codes(latitudes, longitudes)

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,48.35380,11.786100


### 4.2 Collect Data
#### 4.2.1 Setup ICAO Dataframe

In [24]:
cities = ['Berlin', 'Hamburg', 'Munich']
icao_codes = ['EDDB', 'EDDH', 'EDDM']

icao_df = pd.DataFrame({
    'City': cities,
    'ICAO': icao_codes
})

icao_df

,City,ICAO
0,Berlin,EDDB
1,Hamburg,EDDH
2,Munich,EDDM


In [25]:
# City ID Column
icao_df['city_id'] = range(1, len(icao_df) + 1)
icao_df

,City,ICAO,city_id
0,Berlin,EDDB,1
1,Hamburg,EDDH,2
2,Munich,EDDM,3


#### 4.2.2 Check Flight Arrivals Citywise
- **Sample Check**: For Berlin
- **Arrival Times**: Sort and Order for Flight Arrivals
- **Automated For Loop Citywise**: for icao, flight number, departure airport, flight status, aircraft info etc.

In [ ]:
#Sample Check Berlin
url2 = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2024-11-28T11:00/2024-11-28T23:00"

querystring = {"withLeg":"false","withCancelled":"true","withCodeshared":"true","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": "Input", # Insert your RapidAPI Key
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = req.get(url2, headers=headers, params=querystring)

arrival_information_berlin = response.json()
arrival_information_berlin

{'departures': [{'movement': {'airport': {'icao': 'GCTS',
     'iata': 'TFS',
     'name': 'Tenerife Island',
     'timeZone': 'Atlantic/Canary'},
    'scheduledTime': {'utc': '2024-11-28 10:00Z',
     'local': '2024-11-28 11:00+01:00'},
    'revisedTime': {'utc': '2024-11-28 10:00Z',
     'local': '2024-11-28 11:00+01:00'},
    'runwayTime': {'utc': '2024-11-28 10:19Z',
     'local': '2024-11-28 11:19+01:00'},
    'terminal': '1',
    'checkInDesk': '611-616',
    'gate': 'A36',
    'runway': '25L',
    'quality': ['Basic', 'Live']},
   'number': 'U2 5113',
   'callSign': 'EJU19RC',
   'status': 'Departed',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'reg': 'OE-IZN', 'modeS': '440CA9', 'model': 'Airbus A320'},
   'airline': {'name': 'easyJet', 'iata': 'U2', 'icao': 'EZY'}},
  {'movement': {'airport': {'icao': 'GCTS',
     'iata': 'TFS',
     'name': 'Tenerife Island',
     'timeZone': 'Atlantic/Canary'},
    'scheduledTime': {'utc': '2024-11-28 10:00Z',
  

In [61]:
# Sort out & Order Flights
# Hash and run one at a time, as needed

pd.json_normalize(arrival_information_berlin['arrivals'])
arrival_information_berlin['arrivals'][0]
arrival_information_berlin['departures'][0]

#Flight Number
arrival_information_berlin['arrivals'][0]['number']

#Status
arrival_information_berlin['arrivals'][0]['status']

#Departure Airport
arrival_information_berlin['arrivals'][0]['movement']['airport']['name']

#Revised arrival time
arrival_information_berlin['arrivals'][0]['movement']['revisedTime']['local']

#Scheduled arrival time
arrival_information_berlin['arrivals'][0]['movement']['scheduledTime']['local']

#Aircraft
arrival_information_berlin['arrivals'][0]['aircraft']['model']

'Airbus A320 NEO'

In [ ]:
# Data Exploration Cities 'Berlin', 'Hamburg', 'Munich'
# ICAO Codes ['EDDB', 'EDDH', 'EDDM']
list_of_airports = ['EDDB']

In [63]:
current_datetime = datetime.now()
time_from = current_datetime.strftime("%Y-%m-%d %H:%M")
time_from

'2024-12-03 16:00'

In [64]:
time_to = current_datetime + timedelta(hours=12)
time_to = time_to.strftime("%Y-%m-%d %H:%M")
time_to

'2024-12-04 04:00'

Code-On: 
Using a `for` loop with `try` and `except KeyError` is useful when iterating over a collection (like a list or dictionary) where some keys might not exist. This approach allows the program to handle missing keys gracefully without crashing, ensuring that the rest of the loop continues to execute.

Here's a brief example in Python:

```python
data = [{'name': 'Alice'}, {'name': 'Bob'}, {'age': 30}]

for item in data:
    try:
        print(item['name'])
    except KeyError:
        print('Name key is missing')
```

In this example, the `try` block attempts to access the 'name' key, and the `except KeyError` block handles cases where the 'name' key is not present, preventing the program from crashing.

In [ ]:
# For Loop for Data Extraction

flight_data = []

for airport in list_of_airports:
    
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{time_from}/{time_to}"
    querystring = {"withLeg":"false","withCancelled":"true","withCodeshared":"true","withLocation":"false"}
    headers = {
        "X-RapidAPI-Key": "Input",  # Insert your RapidAPI Key
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}

    response = req.get(url, headers=headers, params=querystring)
    flight_json = response.json()

    for entry in flight_json['arrivals']:
        try:
            revised_time = entry['movement']['revisedTime']['local']
        except KeyError:
            revised_time = 'N/A' 

        flight_data.append({
            "ICAO": airport,
            "Flight Number": entry['number'],
            "Departure Airport": entry['movement']['airport']['name'],
            "Status": entry['status'],
            "Scheduled arrival date_time": entry['movement']['scheduledTime']['local'],
            "Revised arrival date_time": revised_time
        })

flight_df = pd.DataFrame(flight_data)
flight_df

,ICAO,Flight Number,Departure Airport,Status,Scheduled arrival date_time,Revised arrival date_time
0,EDDB,LH 1942,Munich,Expected,2024-12-03 16:00+01:00,2024-12-03 16:00+01:00
1,EDDB,TP 532,Lisbon,Expected,2024-12-03 16:10+01:00,2024-12-03 16:00+01:00
2,EDDB,PC 985,Istanbul,Expected,2024-12-03 16:15+01:00,2024-12-03 16:09+01:00
3,EDDB,6H 253,Tel Aviv Yafo,Expected,2024-12-03 16:35+01:00,2024-12-03 16:14+01:00
4,EDDB,OS 231,Vienna,Expected,2024-12-03 16:20+01:00,2024-12-03 16:20+01:00
...,...,...,...,...,...,...
112,EDDB,FR 2419,Tenerife Island,Expected,2024-12-03 23:00+01:00,2024-12-03 23:00+01:00
113,EDDB,U2 8631,London,Expected,2024-12-03 23:00+01:00,2024-12-03 23:00+01:00
114,EDDB,EC 8631,London,Expected,2024-12-03 23:00+01:00,2024-12-03 23:00+01:00
115,EDDB,I2 1809,Madrid,Expected,2024-12-03 23:10+01:00,2024-12-03 23:10+01:00


#### 4.2.3 Collect Data
Automating a function to collect data with a flist as below

In [18]:
# Part 1: Dive Deeper in 3 parts
list_of_airports = ['EDDB', 'EDDH', 'EDDM']

In [ ]:
def get_flight_data(list_of_airports):
    
    flight_data = []
    
    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    tomorrow = (today + timedelta(days=1))

    for airport in list_of_airports:
        
        times = [["00:00","11:59"],["12:00","23:59"]]

        for time in times:
            
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"withLeg":"false","withCancelled":"true","withCodeshared":"true","withLocation":"false"}
            headers = {
                "X-RapidAPI-Key": "Input",  # Insert your RapidAPI Key
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}

            response = req.get(url, headers=headers, params=querystring)
            flight_json = response.json()

            for entry in flight_json['arrivals']:
                try:
                    revised_time = entry['movement']['revisedTime']['local']
                except KeyError:
                    revised_time = 'N/A' 
                    
                try:
                    aircraft_model = entry['aircraft']['model']
                except KeyError:
                    aircraft_model = 'N/A'

                flight_data.append({
                    "ICAO": airport,
                    "Flight_Number": entry['number'],
                    "Departure_Airport": entry['movement']['airport']['name'],
                    "Flight_Status": entry['status'],
                    "Scheduled_arrival_date": entry['movement']['scheduledTime']['local'],
                    "Revised_arrival_date": revised_time,
                    "Aircraft": aircraft_model,
                    "Data_retrieved_on": today
                })

    flight_df = pd.DataFrame(flight_data)
    return flight_df

In [17]:
flight_df = get_flight_data(list_of_airports)
flight_df

,ICAO,Flight_Number,Departure_Airport,Flight_Status,Scheduled_arrival_date,Revised_arrival_date,Aircraft,Data_retrieved_on
0,EDDB,FI 518,Reykjavik,Expected,2024-12-04 06:15+01:00,N/A,Boeing 737,2024-12-03
1,EDDB,HU 489,Beijing,Expected,2024-12-04 06:40+01:00,2024-12-04 06:40+01:00,Boeing 787-9,2024-12-03
2,EDDB,QR 79,Doha,Expected,2024-12-04 06:55+01:00,2024-12-04 06:55+01:00,Boeing 787-9,2024-12-03
3,EDDB,EW 8001,Stuttgart,Expected,2024-12-04 07:35+01:00,2024-12-04 07:35+01:00,Airbus A319,2024-12-03
4,EDDB,EW 2,Cologne,Expected,2024-12-04 07:40+01:00,2024-12-04 07:40+01:00,Airbus A320-200 (sharklets),2024-12-03
...,...,...,...,...,...,...,...,...
776,EDDM,LO 355,Warsaw,Expected,2024-12-04 22:25+01:00,2024-12-04 22:25+01:00,Boeing 737,2024-12-03
777,EDDM,IB 747,Madrid,Expected,2024-12-04 22:40+01:00,2024-12-04 22:40+01:00,Airbus A320,2024-12-03
778,EDDM,BA 958,London,Expected,2024-12-04 22:45+01:00,2024-12-04 22:45+01:00,Airbus A320,2024-12-03
779,EDDM,LH 2483,London,Expected,2024-12-04 23:05+01:00,2024-12-04 23:05+01:00,Airbus A320 NEO,2024-12-03


In [19]:
# Part 2: Cleaning the Dataframe
def clean_flight_data(flight_df):
    
    flight_df['Scheduled_arrival_date'] = pd.to_datetime(flight_df['Scheduled_arrival_date'])
    flight_df['Data_retrieved_on'] = pd.to_datetime(flight_df['Data_retrieved_on'])
    flight_df['Revised_arrival_date'] = pd.to_datetime(flight_df['Revised_arrival_date'], errors='coerce')
    
    return flight_df


In [20]:
# Part 3: Setting up Final Cleaned Data
flight_df_final = clean_flight_data(flight_df)
flight_df_final

,ICAO,Flight_Number,Departure_Airport,Flight_Status,Scheduled_arrival_date,Revised_arrival_date,Aircraft,Data_retrieved_on
0,EDDB,FI 518,Reykjavik,Expected,2024-12-04 06:15:00+01:00,NaT,Boeing 737,2024-12-03
1,EDDB,HU 489,Beijing,Expected,2024-12-04 06:40:00+01:00,2024-12-04 06:40:00+01:00,Boeing 787-9,2024-12-03
2,EDDB,QR 79,Doha,Expected,2024-12-04 06:55:00+01:00,2024-12-04 06:55:00+01:00,Boeing 787-9,2024-12-03
3,EDDB,EW 8001,Stuttgart,Expected,2024-12-04 07:35:00+01:00,2024-12-04 07:35:00+01:00,Airbus A319,2024-12-03
4,EDDB,EW 2,Cologne,Expected,2024-12-04 07:40:00+01:00,2024-12-04 07:40:00+01:00,Airbus A320-200 (sharklets),2024-12-03
...,...,...,...,...,...,...,...,...
776,EDDM,LO 355,Warsaw,Expected,2024-12-04 22:25:00+01:00,2024-12-04 22:25:00+01:00,Boeing 737,2024-12-03
777,EDDM,IB 747,Madrid,Expected,2024-12-04 22:40:00+01:00,2024-12-04 22:40:00+01:00,Airbus A320,2024-12-03
778,EDDM,BA 958,London,Expected,2024-12-04 22:45:00+01:00,2024-12-04 22:45:00+01:00,Airbus A320,2024-12-03
779,EDDM,LH 2483,London,Expected,2024-12-04 23:05:00+01:00,2024-12-04 23:05:00+01:00,Airbus A320 NEO,2024-12-03


In [21]:
flight_df_sql=flight_df_final.copy()
flight_df_sql

,ICAO,Flight_Number,Departure_Airport,Flight_Status,Scheduled_arrival_date,Revised_arrival_date,Aircraft,Data_retrieved_on
0,EDDB,FI 518,Reykjavik,Expected,2024-12-04 06:15:00+01:00,NaT,Boeing 737,2024-12-03
1,EDDB,HU 489,Beijing,Expected,2024-12-04 06:40:00+01:00,2024-12-04 06:40:00+01:00,Boeing 787-9,2024-12-03
2,EDDB,QR 79,Doha,Expected,2024-12-04 06:55:00+01:00,2024-12-04 06:55:00+01:00,Boeing 787-9,2024-12-03
3,EDDB,EW 8001,Stuttgart,Expected,2024-12-04 07:35:00+01:00,2024-12-04 07:35:00+01:00,Airbus A319,2024-12-03
4,EDDB,EW 2,Cologne,Expected,2024-12-04 07:40:00+01:00,2024-12-04 07:40:00+01:00,Airbus A320-200 (sharklets),2024-12-03
...,...,...,...,...,...,...,...,...
776,EDDM,LO 355,Warsaw,Expected,2024-12-04 22:25:00+01:00,2024-12-04 22:25:00+01:00,Boeing 737,2024-12-03
777,EDDM,IB 747,Madrid,Expected,2024-12-04 22:40:00+01:00,2024-12-04 22:40:00+01:00,Airbus A320,2024-12-03
778,EDDM,BA 958,London,Expected,2024-12-04 22:45:00+01:00,2024-12-04 22:45:00+01:00,Airbus A320,2024-12-03
779,EDDM,LH 2483,London,Expected,2024-12-04 23:05:00+01:00,2024-12-04 23:05:00+01:00,Airbus A320 NEO,2024-12-03


### 4.3 Flight Data
#### 4.3.1 Config & Dataframe
You may also use the config file provided in the Github Repository to run this (**Option 1**)

In [22]:
# Option 1: Using the Alternate Method to Get Config Data
from safe import safe
safe.get('SERVER_HOST')
safe.get('SERVER_USER')
safe.get('SERVER_PASSWORD')
safe.get('SERVER_PORT')
safe.get('FLIGHT_API_KEY')

In [ ]:
# Option 2: Without safe.py file
schema = "Input"  # Insert your schema name
host = "Input"  # Insert your host
user = "Input"       # Insert your user  
password = 'Input' # Insert your password
port = Input     # Insert your port

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### 4.2 Flight Dataframe Creation
#### 4.2.2 SQL Push-Pull Request

In [28]:
flight_df_sql.to_sql('flight_data',
              if_exists='append',
              con=connection_string,
              index=False)

781

### 5. Retrospection
##### 1. Flights Data
- **Challenges**: 
    - **API Rate Limiting**: Handling the rate limits imposed by the AeroDataBox API, which required careful timing and batching of requests to avoid hitting the limit.
    - **Data Consistency**: Ensuring consistent data retrieval, especially when dealing with missing keys or values in the API response, which required robust error handling.
    - **Timezone Management**: Managing timezones correctly, particularly when converting and comparing datetime values from different sources to ensure accurate scheduling and arrival times.
   
- **Highlights**:
    - **Automated Data Collection**: Successfully automated the process of collecting flight data for multiple airports, which streamlined the data retrieval process and reduced manual effort.
    - **Data Cleaning and Transformation**: Implemented effective data cleaning and transformation functions to ensure the data was in a suitable format for analysis and storage in the SQL database.
    - **SQL Integration**: Seamlessly integrated the cleaned flight data into a MySQL database, enabling efficient storage and retrieval for further analysis and reporting.
